In [49]:
import jsonlines
TEST_TASK = 101

java_dataset = []
with jsonlines.open('../MP3/selected_humanevalx_java_248264623217713294771470079546488913479.jsonl', 'r') as reader:
    for line in reader: 
        java_dataset.append(line)
java_entry = [java_entry for java_entry in java_dataset if java_entry["task_id"] == "HumanEval/"+str(TEST_TASK)][0]
# print(java_entry)

dataset = []
with jsonlines.open('../MP3/selected_humanevalx_python_248264623217713294771470079546488913479.jsonl', 'r') as reader:
    for line in reader: 
        dataset.append(line)
        
entry = [entry for entry in dataset if entry["task_id"] == "HumanEval/"+str(TEST_TASK)][0]
print(entry['declaration'])
print(entry['canonical_solution'])
print("--------------------")

print(java_entry['declaration'])
print(java_entry['canonical_solution'])

def words_string(s):

    if not s:
        return []

    s_list = []

    for letter in s:
        if letter == ',':
            s_list.append(' ')
        else:
            s_list.append(letter)

    s_list = "".join(s_list)
    return s_list.split()

--------------------
import java.util.*;
import java.lang.*;

class Solution {
    public List<String> wordStrings(String s) {

        if (s.length() == 0) {
            return List.of();
        }
        StringBuilder sb = new StringBuilder();

        for (char letter : s.toCharArray()) {
            if (letter == ',') {
                sb.append(' ');
            } else {
                sb.append(letter);
            }
        }

        return new ArrayList<>(Arrays.asList(sb.toString().split("\s+" )));
    }
}


In [50]:
response = """
```java
import java.util.Arrays;
import java.util.List;
import java.util.stream.Collectors;

public class Main {
    public static void main(String[] args) {
        System.out.println(wordsString("Hi, my name is John"));
        System.out.println(wordsString("One, two, three, four, five, six"));
    }

    public static List<String> wordsString(String s) {
        if (s == null || s.isEmpty()) {
            return Arrays.asList();
        }

        StringBuilder sb = new StringBuilder();
        for (char c : s.toCharArray()) {
            if (c == ',') {
                sb.append(' ');
            } else {
                sb.append(c);
            }
        }

        return Arrays.stream(sb.toString().split(" "))
                .collect(Collectors.toList());
    }
}
```
"""

In [51]:
import re

# Find the Java code in the model response between [Java Start] and [Java End]
# java_raw = re.search(r"\[Java Start\](.*?)\[Java End\]", response, re.DOTALL).group(1) 
# DELIMS = ['[Java Start]', '[Java End]']
DELIMS = ['```java', '```']
matches = re.findall(rf"{re.escape(DELIMS[0])}(.*?){re.escape(DELIMS[1])}", response, re.DOTALL)

java_raw = ''
if matches:
    java_raw = matches[-1]  # Get the last match
# print(java_raw)

# Combine the imports from the response and canonical
# We don't care if the model gets imports wrong
java_imports = re.findall(r"(import .*?;)", java_raw)
canonical_imports = re.findall(r"(import .*?;)", java_entry['declaration'])
combined_imports = '\n'.join(list(set(java_imports + canonical_imports)))

# Assemble the new declaration with the combined imports
# modified_declaration = combined_imports+re.sub(r"(import .*?;)", "", java_entry['declaration'])
modified_declaration = combined_imports+'\n\nclass Solution {\n'

# Extract the methods from the model response
methods = []
method_count = -1
method_braces_count = 0
in_method = False

for j in java_raw.split('\n'):
    if ('public' in j or 'private' in j) and 'class' not in j:
        method_count += 1
        methods.append('')
        method_braces_count = 0
        in_method = True
    if in_method:
        methods[method_count] += j + '\n'
        method_braces_count += j.count('{') - j.count('}')
        if method_braces_count == 0:
            in_method = False
        
# Filter out main method and any classes 
# We don't care if the model tries to provide a main method or solution class
methods = [m for m in methods if ('Main' not in m) and ('main' not in m) and ('class' not in m)]

# Assemble the final java code from the pieces
java_extract = modified_declaration 
java_extract += ''.join(methods) + '\n}\n' # Close the Solution class
java_extract += java_entry["test"] 
print(java_extract)


import java.util.*;
import java.util.Arrays;
import java.lang.*;
import java.util.stream.Collectors;
import java.util.List;

class Solution {
    public static List<String> wordsString(String s) {
        if (s == null || s.isEmpty()) {
            return Arrays.asList();
        }

        StringBuilder sb = new StringBuilder();
        for (char c : s.toCharArray()) {
            if (c == ',') {
                sb.append(' ');
            } else {
                sb.append(c);
            }
        }

        return Arrays.stream(sb.toString().split(" "))
                .collect(Collectors.toList());
    }

}
public class Main {
    public static void main(String[] args) {
        Solution s = new Solution();
        List<Boolean> correct = Arrays.asList(
                s.wordStrings("Hi, my name is John" ).equals(Arrays.asList("Hi", "my", "name", "is", "John" )),
                s.wordStrings("One, two, three, four, five, six" ).equals(Arrays.asList("One", "two", "three", "four", 

In [52]:
with open('Main.java', 'w') as f:
    f.write(java_extract)

In [58]:
import subprocess
command1 = ['javac', 'Main.java']
command2 = ['java', 'Main']
compile_result = subprocess.run(command1, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
run_result = subprocess.run(command2, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(compile_result, run_result)
print('----')
java_ouput = f"""
               For task {entry["task_id"]}:
               Compile rc     = {str(compile_result.returncode)} 
               Compile stdout = {compile_result.stdout.decode('utf-8')} 
               Compile stderr = {compile_result.stderr.decode('utf-8')} 
               
               Run rc         = {str(run_result.returncode)}
               Run stdout     = {run_result.stdout.decode('utf-8')}
               Run stderr     = {run_result.stderr.decode('utf-8')}
               """
             

print(java_ouput)

CompletedProcess(args=['javac', 'Main.java'], returncode=0, stdout=b'', stderr=b'') CompletedProcess(args=['java', 'Main'], returncode=0, stdout=b'', stderr=b'')
----

               For task HumanEval/101:
               Compile rc     = 0 
               Compile stdout =  
               Compile stderr =  
               
               Run rc         = 0
               Run stdout     = 
               Run stderr     = 
               
